<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Linear Regression

_Authors: Matt Brems (DC), Marc Harper (LA)_

---

<a id="learning-objectives"></a>
### Learning Objectives 
- Understand modeling, inference from models, and prediction
- Understand the framework of regression modeling
- Understand simple linear regression and linear relationships between variables
- Learn how to find the best slope and intercept for simple linear regression
- Be able to interpret the results of simple linear regression
- Understand the assumptions that simple linear regression makes

### Lesson Guide

- [Naive/Baseline Prediction](#naivebaseline-prediction)
- [Finding the "best" slope and intercept](#finding-the-best-slope-and-intercept)
- [Interpretation of SLR](#interpretation-of-slr)
- [Assumptions of SLR](#assumptions-of-slr)
- [Final notes on linear regression](#final-notes-on-linear-regression)
- [Categorical predictors](#categorical-predictors)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

<a id="naivebaseline-prediction"></a>
## Naive/Baseline Prediction

What is the best prediction we can make for any data point if we can only use a single value?

How can we measure the success of our prediction?

<a id="lets-generate-some-data"></a>
### Let's generate some data

In [ ]:
# create an x and a y that is 
# 50 plus 2 times x with some random noise around that
np.random.seed(42)


If we know very little about our data and can only use a single value, we can still predict the mean.

In [ ]:
### Plot the data
# create a figure


# get current axis of that figure


# plot a scatter plot on it with our data


We'll add that prediction to our DataFrame to use later

<a id="finding-the-best-slope-and-intercept"></a>
## Finding the "best" slope and intercept

---

**1. If we want to find the "best" slope an intercept, we will need a criterion that defines what it means to be the "best".**

**2. We need to be able to numerically optimize (maximize or minimize) this criterion.**


<a id="residuals"></a>
### Residuals

The difference between the true value of $Y$ and the predicted value of $Y$ (denoted $\hat{Y}$, "Y-hat") is known as a "residual." We calculate a residual as $Y_i - \hat{Y}_i$ and denote each residual as $\varepsilon_i$.

**Note that the residual $\varepsilon_i$ is the same as the error in our modeled linear relationship!**

For every value of $i$ from 1 to $n$, we:
- have an observed $X_i$.
- have an observed $Y_i$.
- can calculate a predicted value $\hat{Y}_i$, which is equal to $f(X_i)$, where $f$ is our model.
- can calculate a residual $\varepsilon_i$, which is equal to $Y_i - \hat{Y}_i$.

<a id="residual-sum-of-squares-rss"></a>
### Residual sum of squares (RSS)

The most common way to define the "best fit line" is one that forces our residuals (a.k.a. errors) will be as close to zero as possible.

In fact, we seek to minimize the *sum of squared residuals* or [residual sum of squares (RSS)](https://en.wikipedia.org/wiki/Residual_sum_of_squares).

### $$RSS = \sum_{i=1}^n (Y_i - \hat{Y}_i)^2 $$

This is also known as a **loss function**, where the "loss" here is the sum of squared residuals.

---

**Review: what is the difference between a parameter and a statistic?**

Below are two equations, one using parameters and one using statistics. What is the difference?

### $$ Y = \beta_0 + \beta_1 X \\
\hat{Y} = \hat{\beta}_0 + \hat{\beta}_1 X $$

In [ ]:
# 
# 

Remember our naive prediction? Let's see how it does.

In [ ]:
# Calculate SSE


In [ ]:
# Calculate MSE


<a id="minimizing-the-mse"></a>
### Minimizing the MSE

For simple linear regression [we can derive the intercept and slope that minimize the RSS algebraically](https://en.wikipedia.org/wiki/Simple_linear_regression#Fitting_the_regression_line). 

Given variables:
- $\bar{y}$ : the sample mean of observed values $Y$
- $\bar{x}$ : the sample mean of observed values $X$
- $s_Y$ : the sample standard deviation of observed values $Y$
- $s_X$ : the sample standard deviation of observed values $X$
- $r_{XY}$ : the sample Pearson correlation coefficient between observed $X$ and $Y$

The equations for the intercept and slope that minimize the RSS is:

### $$ \hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x} \\
\hat{\beta}_1 = r_{XY} \frac{s_Y}{s_X}$$

Our simple linear regression is an **estimator of the expected value (mean) of $Y$**.

> **Note:** Estimators are mathematical functions or formulas that yield an *estimate*. For example, an estimator might be the sample mean whereas the estimate is the value of the sample mean from our data set.

<a id="calculate-beta-coefficients-by-hand"></a>
### Calculate beta coefficients (a.k.a. "fit" model) by hand

In [ ]:
y_bar = 
x_bar = 
std_y = 
std_x = 
r_xy = 

beta_1 = 
beta_0 = 

In [ ]:
# predict with model
df['Linear_Yhat'] = beta_0 + beta_1 * df['X']

In [ ]:
# calculate MSE
np.square(df['Y'] - df['Linear_Yhat']).mean()

**How does our new MSE compare to our old MSE?**

In [ ]:
# 

#### We can now plot our line

#### We can also plot our residuals

In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
fig.set_figheight(8)
fig.set_figwidth(15)

# get the axis
ax = fig.gca()

# plot it
ax.scatter(x=df['X'], y=df['Y'], c='k')
ax.plot(df['X'], df['Linear_Yhat'], color='k');

# iterate over predictions
for _, row in df.iterrows():
    plt.plot((row['X'], row['X']), (row['Y'], row['Linear_Yhat']), 'r-')

<a id="interpretation-of-slr"></a>
## Interpretation of SLR

---

Suppose I believe that commute time, my $Y$, is best modeled as a linear function of the number of Metro stops I take. Then, my model is:
### $$ [commute] = \beta_0 + \beta_1 [Metro\_stops] $$

I want to estimate this model from our data. I gather data over a long period of time and put it into a Pandas dataframe with a column for commute time and a column for number of Metro stops. Then, my estimate of the true model is:

### $$ \hat{[commute]} = \hat{\beta}_0 + \hat{\beta}_1 [Metro\_stops] $$

Let's say that $\hat{\beta}_0$ is 5 and $\hat{\beta}_1$ is 3.
- I interpret $\hat{\beta}_1$ by saying "As the number of Metro stops increases by 1, I expect commute time to increase by three minutes."
- I interpret $\hat{\beta}_0$ by saying "If someone travels zero stops on the Metro, I expect commute time to be five minutes."

The simple linear regression estimators described above have really nice properties:

1. They are unbiased. (a.k.a. the statistics will on average equal the true parameter value)
2. Among all unbiased estimators, the estimators above have the smallest possible variance.
3. They are consistent. (a.k.a. as your sample size n goes to infinity, the statistic converges to the true parameter value)

**However, these are just mathematical properties.** They do not take into account real-world situations. For example, suppose I want to use a SLR model to predict my commute time and I get the model:

    [commute time] = 8 * [number of Metro stops] - 10

In this case, my commute time is estimated to be negative if I go zero stops or one stop on the Metro. If I were to interpret my intercept, I might say "If I travel zero Metro stops on my way to work, I expect to commute -10 minutes." Of course, this is nonsense; you can't have a negative commute time!

**What this means:** 

When building a model, make sure your model makes sense! You are responsible for the interpretation of the model. All the computer will do is optimize. You may not do anything wrong when you fit a model (and you won't get an error!), but make a note that there are shortcomings to the framework.

<a id='assumptions'></a>

<a id="assumptions-of-slr"></a>
## Assumptions of SLR

---

There are some critical assumptions involved in SLR that you must be aware of:

1. **Linearity:** Y and X must have an approximately linear relationship.
2. **Independence:** Errors (residuals) $\varepsilon_i$ and $\varepsilon_j$ must be independent of one another for any $i \neq j$.
3. **Normality:** The errors (residuals) follow a Normal distribution with mean 0.
4. **Equality of Variances** (Homoscedasticity of errors): The errors (residuals) should have a roughly consistent pattern, regardless of the value of X. (There should be no discernable relationship between X and the residuals.)

The mnemonic **LINE** is a useful way to remember these four assumptions.

Lengthier discussions of the four main assumptions and how one can evaluate or test for each of these [can be found here](http://people.duke.edu/~rnau/testing.htm).

If all four assumptions are true, the following holds: 

### $$ Y_i \sim N(\beta_0 + \beta_1 X_i, \sigma) $$

# Multiple Linear Regression

<a id='load-data'></a>

## Load the diamonds data

---

Over the course of this lesson we will be constructing a simple linear regression and then extend this to multiple linear regression. Included in the datasets folder is a dataset on diamond prices.

**Load the data using pandas.**

In [ ]:
diamonds_csv = './datasets/diamonds.csv'

The columns are:

    carat: the carat weight of diamonds
    cut, color, clarity: (ignore)
    depth, table, x, y, z: diamond measurements
    price: the price of the diamond

<a id='slr-to-mlr'></a>

## From simple linear regression (SLR) to multiple linear regression (MLR)

---

The TL;DR of multiple linear regression:

> Instead of using just one predictor to estimate a continuous target, we build a model with multiple predictor variables. You will be using MLR way more than SLR going forward.

These variables will be represented as columns in a matrix (often a pandas dataframe).

**Brainstorm some examples of real-world scenarios where multiple predictors would be beneficial. Can you think of cases where it might be detrimental?**

<a id='assumptions'></a>

## Assumptions of MLR

---

Like SLR, there are assumptions associated with MLR. Luckily, they're quite similar to the SLR assumptions.

1. **Linearity:** $Y$ must have an approximately linear relationship with each independent $X_i$.
2. **Independence:** Errors (residuals) $\varepsilon_i$ and $\varepsilon_j$ must be independent of one another for any $i \ne j$.
3. **Normality:** The errors (residuals) follow a Normal distribution with mean 0.
4. **Equality of Variances**: The errors (residuals) should have a roughly consistent pattern, regardless of the value of the $X_i$ predictors. (There should be no discernable relationship between the $X$ predictors and the residuals.)
5. **Independence of Predictors**: The independent variables $X_i$ and $X_j$ must be independent of one another for any $i \ne j$.

The mnemonic LINEI is a useful way to remember these five assumptions.

<a id='fit-mlr'></a>

## Fitting a multiple linear regression

---

The formula for computing the $\beta$ values in multiple regression is best done using linear algebra. We will cover the derivation, but for more depth  [these slides are a great resource](https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf).

$X$ is now a _matrix_ of predictors $x_1$ through $x_i$ (with each column a predictor), and $y$ is the target vector we are seeking to estimate. There is still only 1 *estimated* variable!

### $$ \hat{y} = X \beta$$

**Note:** $\beta$ in the formula above is a *vector* of coefficients now, rather than a single value.

In different notation we could write $\hat{y}$ calculated with:

### $$ \hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n $$

---

<a id='mlr-beta-derivation'></a>

### Deriving the $\beta$ coefficients

$\beta$ is solved with the linear algebra formula:

### $$ \beta = (X^TX)^{-1}X^Ty $$

Where $X'$ is the transposed matrix of original matrix $X$ and $(X^TX)^-1$ is the inverted matrix of $X^TX$.



The equation using true $y$ is:

### $$ y = X \beta + \varepsilon $$

Again, $\varepsilon$ is our vector of errors, or residuals.

We can equivalently formulate this as in terms of the residuals:

### $$ \varepsilon = X \beta - y $$

Our goal is to minimize the sum of squared residuals. The sum of squared residuals is equivalent to the dot product of the vector of residuals:

### $$ \sum_{i=1}^n \varepsilon_i^2 = 
\left[\begin{array}{cc}
\varepsilon_1 \cdots \varepsilon_n
\end{array}\right] 
\left[\begin{array}{cc}
\varepsilon_1 \\ \cdots \\ \varepsilon_n
\end{array}\right] = \varepsilon^T \varepsilon
$$

Therefore we can write the sum of squared residuals as:

### $$ \varepsilon^T \varepsilon = (X \beta - y)^T (X \beta - y) $$

Which becomes:

### $$ \varepsilon^T \varepsilon = y^Ty - y^TX\beta - \beta^T X^T y + \beta^T X^T X \beta $$

Now take the derivative with respect to $\beta$:

### $$ \frac{\partial \varepsilon^T \varepsilon}{\partial \beta} = 
-2X^Ty + 2X^TX\beta$$

We want to minimize the sum of squared errors, and so we set the derivative to zero and solve for the beta coefficient vector:

### $$ 0 = -2X^Ty + 2X^TX\beta \\
X^TX\beta = X^Ty \\
\beta = (X^TX)^{-1}X^Ty$$

<a id='code-mlr'></a>

### Code a MLR

**First, we need to create the "design matrix" of our predictors.**

The first column will be a column of all 1s (the intercept) and the other columns will be `carat`, `depth`, `table`, `x`, `y`, `z`.

This is easiest to do with pandas: add a column for the intercept first, then extract the matrix using with `.values`.

In [ ]:
diamonds.head()

### Solve for the beta coefficients

We are still predicting `price`. Implement the linear algebra equation to solve for the beta coefficients. 

### $$ \beta = (X^TX)^{-1}X^Ty $$

**Tips:**

The transpose of a matrix is calculated by appending `.T` to the matrix:

    X.T

Matrices multipled in the formula should be done with the "dot product":

    np.dot(mat1, mat2)

Inverting a matrix is done using:

    np.linalg.inv()

**Confirm that these betas are the same as the ones using `sklearn.linear_model.LinearRegression`**

```python
from sklearn.linear_model import LinearRegression

linreg = LinearRegression(fit_intercept=False)
linreg.fit(X, price)

print(linreg.coef_)
```

**Calculate predicted $\hat{y}$ with your $X$ predictor matrix and $\beta$ coefficients.**

In [ ]:
yhat = 

**Calculate the RMSE of the multiple regression model.**

In [ ]:
print('Multiple regression MSE:', np.sqrt(np.mean(np.square(price - yhat))))

<a id='additional-resources'></a>

## Additional resources

---

[Maximum likelihood estimation](https://onlinecourses.science.psu.edu/stat504/node/28)

<a id='intro'></a>

##  `sklearn` & `statsmodels`

---

This lesson intends to introduce the modeling packages `sklearn` and `statsmodels` in the context of regression modeling. These are both powerful python packages with different strengths. 

In general:
- **`sklearn`** is the *machine learning* package
- **`statsmodels`** is the *statistics* package

Though the terms have immense overlap, machine learning tends to be more prediction focused while statistics is more inference focused. 

**Remember: even with all the power provided by these modeling tools, it's never a replacement for good EDA!**

---

### A preface on modeling

As we venture down the path of modeling, it can be difficult to determine which choices are "correct" or "incorrect".  A primary challenge is to understand how different models will perform in different circumstances and different types of data. It's essential to practice modeling on a variety of data.

As a beginner it is essential to learn which metrics are important for evaluating your models and what they mean. The metrics we evaluate our models with inform our actions.  

*Exploring datasets on your own with the skills and tools you learn in class is highly recommended!*

<a id='documentation'></a>

## `sklearn` and `statsmodels` documentation

---

Get familiar with looking up things in the `sklearn` and `statsmodels` documentation. You are going to be doing a lot of it over the course of DSI and beyond.

[The statsmodels documentation can be found here.](http://statsmodels.sourceforge.net/devel/) Many recommend using the bleeding-edge version of statsmodels. [For that you can reference the code on github.](https://github.com/statsmodels/statsmodels/)

[The sklearn documentation can be found here.](http://scikit-learn.org/stable/documentation.html)

The packages have fairly different approaches and syntax for constructing models. Below are examples for linear regression in each package:
* [Linear regression in statsmodels](http://statsmodels.sourceforge.net/devel/examples/#regression)
* [Linear regression in scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

If you haven't yet, familliarize yourself with the format of the documentation.

<a id='sklearn-background'></a>

## Background: Scikit-learn / sklearn

---

<img src="https://avatars1.githubusercontent.com/u/25111?v=3&s=200" style="float: left; margin: 0 25px;"> Scikit-learn was founded in 2007 as a Google summer of code project by [David Cournapeau](https://github.com/cournape).  Later in 2007, Matthieu Brucher published his thesis on Scikit-learn.  Since then, the Scikit-learn project has taken on a worldwide team of owners. A great high level overview of the project can be found in a 2011 publication in [Journal of Machine Learning Research 12 (2011) 2825-2830](http://www.jmlr.org/papers/volume12/pedregosa11a/pedregosa11a.pdf).

Skikit-learn is part of the Scipy family of "kits".  Explore some of the [other projects](https://scikits.appspot.com/scikits) in this family.
<br clear="all"><br>


**Scikit-learn provides a wide variety of machine learning models, including:**

- Linear regression
- Logistic regression
- Support Vector Machines
- Classification And Regression Tree Models
- Naive Bayes
- Clustering Models (K-Means, Hierarchical, DBScan)

**It also handles the construction of typical machine learning pipeline utilities for:**
- Model evaluation
- Model selection
- Preprocessing
- Natural Language Processing
- Dimensionality Reduction

**Scikit-learn comes with a ton of datasets that are cleaned and formatted to work with the models provided by their library:**
- Boston Housing
- Iris Flowers
- Diabetes Diagnostics
- Various sample images (for classification)
  - Faces
  - MNIST (handwriting examples)
- Random data generators
- Spam examples
- Newsgroup classfication

[Read more about Scikit-learn datasets](http://scikit-learn.org/stable/datasets/)

---

### `sklearn` under the hood

- **Numpy**:  The base for data structures and transformations. Input data is represented as numpy arrays, integrating seamlessly with other scientific Python libraries. Numpy’s viewbased memory model limits copies, even when binding with compiled code.  It also provides basic arithmetic and linear algebra operations.<br><br>

- **Scipy**:  Efficient algorithms for linear algebra, sparse matrix representation, special functions and basic statistical functions.<br><br>

- **Cython**:  A language for combining C with Python. Cython makes it easy to reach the performance of compiled languages with Python-like syntax and high-level operations.

<a id='sklearn-first-steps'></a>

## First steps with `sklearn`: loading the data

---

We will fit a linear regression model on the data with `scikit-learn`.

In [ ]:
from matplotlib import pyplot as plt

from sklearn import linear_model
from sklearn.metrics import mean_squared_error

### Putting the data in pandas for convenience

Our target is what we are predicting.  Sometimes this is called the **response variable**.

The target and the data are what we use to train, or **fit** the model with.

In [ ]:
import numpy as np
import pandas as pd

# Create dataframe of main data to use as predictors (later). AKA "X"
df =

# target vector (price)
target =

print(target.shape)
print(df.shape)
df.head()

Our training set is a matrix / dataframe with six variables. We have **6** predictors with **53940** rows/observations.

Our target is a vector that represents a single variable (**price**), which has exactly the same number of observations as our training set: **53940**.

> _Training (fit) and target datasets must always match in length!_

In [ ]:
target.head()

### Index correspondence between target and predictors

Row 0 of **`df`**, our training data, is:

```
[0.23	61.5	55.0	3.95	3.98	2.43]
```

This corresponds to the 0 index observation in our target vector:
```
326
```

These two separate datasets (a matrix/dataframe, and a vector), are what we will use in the `.fit(predictors, target)` function in `sklearn`'s models.  

- The training data is 2D with dimensions: `n_samples x n_features`
- The response is 1D with dimensions: `n_samples`, matching the `n_samples` of the training data.

<a id='first-model-sklearn'></a>

## Fitting our first model with `sklearn`

---

Now let's fit a linear regression model with the housing data. 

First let's visually identify some predictors that seem to have a relationship with price. 

**Plot each variable in `df` against the `target` variable.**

> _Note: If for some reason scikit-learn crashes the jupyter notebook, have conda remove mkl (there's an issue with the newer build on some systems)._

In [ ]:
df.head()

In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on CARAT with our data

In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on DEPTH with our data

In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on TABLE with our data

In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on X with our data


In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on Y with our data


In [ ]:
# create a figure
fig = plt.figure(figsize=(15,7))
# get the axis of that figure
ax = plt.gca()

# plot a scatter plot on Z with our data


**Below we fit a linear regression model predicting `price` (the target vector) from `carat`.**

> **Note:** sklearn models expect the predictor matrix to be 2D and the target to be 1D.

**Make predictions for the X matrix using `.predict(X)`, and score the model ($R^2$) using `model.score(X, y)`.**

Plot the predicted values against the true values of the target, and print the model $R^2$.

> **`.score(predictors, target)`**: a class method / function that returns the coefficient of determination R^2 of the prediction (for regression models).  Found in many models in scikit-learn (but not all).

In [ ]:
predictions  =  model.predict(X)
score        =  model.score(X, y)

# Plot the model

**What can this plot tell us about the model?**

<a id='model-attributes'></a>

## sklearn model class attributes

---

After you run `.fit()`, a sklearn model object often contains a variety of calculated metrics, coefficients, and other information. Which metrics and attributes are present will depend on the model – consult the documentation for specifics. 

Attributes in the `LinearRegression` object include:
- **`.coef_`**: property containing the coeffients for the predictor variables
- **`.intercept_`**: value of the intercept

**Print out the beta coefficient and intercept for the model.**


**What does the coefficient mean in the context of your model?**

<a id='common-metrics'></a>

## Review: common metrics for evaluating regression models

---

The [root mean squared error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation) is a standard measure of model performance. It is the square root of the mean of the sum of squared residuals:

### $$ \operatorname{RMSE}= \sqrt{\frac{1}{n}\sum_{i=1}^n(\hat{y_i} - y_i)^2} $$

The smaller the root mean squared error, the better your model fits the data. 

You are already familiar with the [coefficient of determination $R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination):

### $$ R^2 = 1 - \frac{SS_{reg}}{SS_{tot}} $$

Where the regression sum of squares is the sum of squared residuals for our model:

$SS_{reg}=\sum_i (\hat{y} -\bar{y})^2$

And the total sum of squares is the sum of squared residuals for the *baseline* model. This is essentially the variance of our target.

$SS_{tot} = \sum_i (y_i-\bar{y})^2$

$R^2$ is the most common metric to evaluate a regression and is the default scoring measure in sklearn. When we cover classification models, the `.score` function instead defaults to accuracy.


**Calculate the RMSE of your model by leveraging `sklearn.metrics.mean_squared_error`.**

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print(np.sqrt(mean_squared_error(target, predictions)))
print(r2_score(target, predictions))

<a id='mlr-sklearn'></a>

## Fit a MLR using sklearn

---

We have fit a simple linear regression predicting `price ~ carat + 1` (where the 1 represents the intercept). Use the same sklearn process and `LinearRegression` model to estimate the target with both `carat` and `table`.

In [ ]:
lm = linear_model.LinearRegression()

X = df[['carat','table']].values
y = target

model = lm.fit(X, y)

predictions  =  model.predict(X)
score        =  model.score(X, y)

# Plot the model
plt.figure(figsize=(8,8))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from CARAT + TABLE - $\hat{y}$")
plt.ylabel("Actual Values PRICE - y")

plt.plot([0, np.max(y)], [0, np.max(y)], c = 'k')

plt.show()

print("score: ", score)

**Print out the coefficients from this MLR model and interpret them.**

In [ ]:
print(model.coef_)

<a id='negative-r2'></a>

## A note on negative $R^2$ values

---

Over the course of this class you will encounter negative $R^2$ values. This may seem impossible, and it is in the standard scenario where we are calculating the $R^2$ score on the data we fit the model with.

However, if you fit your model on one sample of data, *then score the model on new data not used to fit the model*, it is possible to end up with negative $R^2$.

**What does it mean to have a negative $R^2$?**

Remember that $R^2$ is 1 minus the error of your regression model divided by the error of the baseline model. A negative $R^2$ means that the regression model is performing *worse* than the baseline model. In the context of fitting our data on one sample of data and scoring on another sample, this means that we would have been better off making predictions on the test sample just using the mean of the target variable in our training set.

We will return to the topic of negative $R^2$ when we talk about training and testing sets and cross-validation.

<a id='statsmodels-intro'></a>

## Fitting a linear regression using `statsmodels`

---

Now we will fit the linear regression model predicting the target from `RM` and `LSTAT`, but this time using `statsmodels`.

The format looks like:

```python
import statsmodels.api as sm

X = df[[['carat','table']].values
# manually add the intercept column:
X = sm.add_constant(X)
y = target

model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict()
```

First we load the statsmodels api module, which contains the ordinary least squares `OLS` model class. The statsmodels process is slightly different:
- We manually make a new column for the intercept in our design matrix $X$.
- The $y$ target variable comes before the $X$ predictor
- The data is provided during the instantiation of the model object, then fit is called without the data.

**Fit the model using statsmodels.**

<a id='statsmodels-summary'></a>

### Statsmodels `.summary()`  function

Once a model is fit with statsmodels, you can print out a variety of summary statistics, metrics, and properties of the model using the `model.summary()` function.

You are already familiar with some of the information available in the summary:
- R-squared
- Number of observations
- Coefficients for the variables and the intercept (const)
- Standard errors of the coefficients, t-statistics, p-values, and confidence intervals

There is also a variety of different metrics that we have not yet talked about. Don't hesitate to look up any of the statistics online if you are curious.

<a id='independent-practice'></a>

## Independent practice

---

Using either scikit-learn or statsmodels (or both, if you prefer), build a model using any set of **continuous** variables of your choice. Evaluate your model using $R^2$. Describe what the $R^2$ means.

<a id='patsy'></a>

## A brief introduction to `patsy` formulas (optional)

---

Why slice and dice the data yourself when you just write a formula that defines your model?

The `patsy` package allows you to specify the construction of your model using a formula string, and then returns the matrices required to fit the model.

Let's say we wanted to predict `target` from `x`, `y` and `z`. We would write a string formula like so:

```
formula = 'target ~ x + y + z'
```

Then, after importing patsy, we can generate our target and predictor matrix by supplying the formula and the dataframe that contains the corresponding columns.

```python
import patsy

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')
```

Note that all of the columns (including the target!) would need to be in `df`.

In [ ]:
import patsy

formula = 'target ~ x + y + z'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

In [ ]:
y.head()

In [ ]:
X.head()

You'll notice that with `return_type='dataframe'` patsy's `.dmatrices()` function returns two pandas dataframes, one for the target and one for the design matrix. 

You'll also notice that it creates an intercept column by default. **If you do not want it to create an intercept column, add a -1 to the formula string**

In [ ]:
formula = 'target ~ x + y + z - 1'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

X.head()

You can then feed in these matrices into statsmodels or sklearn. It is generally a good practice to convert your target matrix into a 1D vector, especially when using sklearn.

> **Tip:** The `.ravel()` function for numpy arrays will "unravel" a multidimensional matrix into a one dimensional vector of values.

In [ ]:
formula = 'target ~ x + y + z'

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

y = y.values.ravel()

model = sm.OLS(y, X).fit()
model.summary()